### Helper Classes

First we get all of our helper modules. The prepare_EMG module will prepare the EMG data for phoneme recognition. The prepare_outputs module will prepare our target labels and align them with our EMG data. The module 'prepare_data' will help us read data from CSV into a dataframe. Finally, 'vis' will help visualize EMG data in both time and frequency domains. 

In [1]:
%load_ext autoreload
%autoreload 2

import prepare_EMG, prepare_outputs, prepare_data, vis

EMG_Prep = prepare_EMG.EMG_preparer()
Output_Prep = prepare_outputs.output_preparer()
Data_Prep = prepare_data.data_preparer()



### Labeling the Data

First, we need to visualize a few EMG voltage graphs to find some sections that most likely contain no subvocalization. Then, we'll need to find some regions that almost certainly do. These two classes of EMG readouts will serve to train an identifier to help us automatically label EMG windows with phonemes. The model used here will most likely be an SVC, inside "prepare_outputs". It will process each EMG window in order, and when it finds one that most likely contains subvocalization, it applies the next phoneme as that window's label. 

In [2]:
data_1 = Data_Prep.load('Sat Mar  4 00:44:23 2017')
data_1[:3000]

data_no_sv = Data_Prep.sv_detection()
data_no_sv

,time,voltage
0,10.0,-4.743750
1,15.0,-5.478516
2,20.0,-6.110156
3,25.0,-6.458203
4,30.0,-6.535547
5,35.0,-6.973828
6,40.0,-7.270312
7,45.0,-7.489453
8,50.0,-7.682812
9,55.0,-7.798828


In [26]:
import pandas 

data_1 = pandas.read_csv('Sat Mar  4 00:44:23 2017')
data_2 = pandas.read_csv('Sat Mar  4 00:45:02 2017')
data_3 = pandas.read_csv('Sat Mar  4 00:45:47 2017')
data_4 = pandas.read_csv('Sat Mar  4 00:47:01 2017')
data_5 = pandas.read_csv('Sat Mar  4 00:47:36 2017')
no_sv = data_1[:3000].append(data_2[:2000]).append(data_3[:2000]).append(data_4[:2000]).append(data_5[:2000])

with_sv = data_1[5000:9000].append(data_2[6000:10000]).append(data_3[3000:9000])

windows = EMG_Prep.process(with_sv)


KeyError: 1

In [21]:
a = 4/2
b = 4//2

a,b

(2.0, 2)